In [ ]:
import pandas as pd

# Load the dataset
data = pd.read_csv('FinalDataset.tsv', sep='\t', on_bad_lines='skip')


In [ ]:
import re

def clean_text(text):
    # Remove any HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    return text

data['review_body'] = data['review_body'].apply(clean_text)


In [ ]:
from sklearn.model_selection import train_test_split

X = data['review_body']
y = data['star_rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Initialize the SVM model
svm_model = SVC(kernel='linear')

# Train the model
svm_model.fit(X_train_tfidf, y_train)

# Make predictions on the test set
y_pred = svm_model.predict(X_test_tfidf)

# Function to map original labels to new categories
def map_sentiment(label):
    if label in [1, 2]:
        return 'negative'
    elif label == 3:
        return 'neutral'
    else:
        return 'positive'

# Map the original labels to the new categories
y_test_mapped = [map_sentiment(label) for label in y_test]
y_pred_mapped = [map_sentiment(label) for label in y_pred]

# Compute and print the confusion matrix
print("\nConfusion Matrix:\n", confusion_matrix(y_test_mapped, y_pred_mapped, labels=['negative', 'neutral', 'positive']))





Confusion Matrix:
 [[ 502   27  188]
 [  42   27  140]
 [  82   21 2960]]


In [ ]:
# Predictions
y_pred = svm_model.predict(X_test_tfidf)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Classification Report
print(classification_report(y_test, y_pred))

# Confusion Matrix
print("\nConfusion Matrix:\n", confusion_matrix(y_test_mapped, y_pred_mapped, labels=['negative', 'neutral', 'positive']))


Accuracy: 75.13%
              precision    recall  f1-score   support

           1       0.65      0.73      0.69       541
           2       0.50      0.05      0.09       176
           3       0.36      0.13      0.19       209
           4       0.41      0.06      0.10       451
           5       0.79      0.97      0.87      2612

    accuracy                           0.75      3989
   macro avg       0.54      0.39      0.39      3989
weighted avg       0.69      0.75      0.69      3989


Confusion Matrix:
 [[ 502   27  188]
 [  42   27  140]
 [  82   21 2960]]


In [ ]:
import numpy as np

# Get feature importance
feature_importance = svm_model.coef_

# Get top 10 important features
top10_indices = np.argsort(feature_importance)[-10:]
top10_features = [tfidf_vectorizer.get_feature_names_out()[i] for i in top10_indices]

print("Top 10 important features:", top10_features)


Top 10 important features: ['09']
